In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import json
import string


In [ ]:
url = 'https://search.sina.com.cn/news'

In [ ]:
referer='https://search.sina.com.cn/news?'

params = {"q": requests.utils.quote("阿里巴巴"), "c": "news", }
data = {"q":"阿里巴巴", "c": "news", }

for key, value in params.items():
    referer += key + '=' + value + '&'


headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded',
    'referer': referer
}

In [ ]:
r = requests.post(url, data=data, headers=headers, allow_redirects=True)

In [ ]:
soup = bs4.BeautifulSoup(r.text, 'html.parser')

In [ ]:
elements = soup.html.find_all(class_=re.compile('box-result'))

def getDate(element):
    headers = {
        'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36',
        'Content-Type': 'application/x-www-form-urlencoded;charset=utf-8',
    }

    detail = requests.get(element.h2.a.get('href'), headers=headers)

    detailSoup = bs4.BeautifulSoup(detail.content.decode('iso-8859-1'), 'html.parser')

    dateElement = detailSoup.find(class_="date")

    text = dateElement.get_text()
    date = re.findall(r'\d+', text)[0:3]

    return '-'.join(date) 

def cleanData(element):
    data = {}
    data['title'] = {'name': '标题', 'value':element.find('h2').text}
    data['url'] = {'name': '链接', 'value': element.find('h2').find('a').get('href')}
    data['source'] ={'name': '来源','value': element.find(class_=re.compile('fgray_time')).get_text().split(' ')[0]}
    data['date'] = {'name': '日期', 'value':getDate(element)}
    return data

file1 = open('data.txt', 'w')
for i in elements:
    res = cleanData(i)

    for key, value in res.items():
        print(value['name'] + ':' + value['value'])
        file1.write(value['name'] + ':' + value['value'] + '\n')


    print('\n')
    file1.write('\n')


